# Spiketrap Homework

Build a strategy to download and store all reddit posts and comments (including upvotes and downvotes) for a given subreddit (eg reddit.com/r/sanfrancisco).

Write down an executable script in any language to run your strategy.

Storage of your choice among Redis, MongoDB, or Mysql. Up to you choose which one you think fits best and/or you are more familiar with.

Tutorial from: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

In [1]:
with open('api_key.txt', 'r') as key_file:
    CLIENT_ID, SECRET_KEY = key_file.read().strip('\n').split('\n')

In [2]:
import requests

In [3]:
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

In [4]:
with open('secret.txt', 'r') as pw_file:
    user, pw = pw_file.read().strip('\n').split('\n')

In [5]:
pw

'crash99dsa33'

In [6]:
user

'Alepv'

In [7]:
user_data = {
    'grant_type': 'password',
    'username': user,
    'password': pw
}

In [8]:
headers = {'User-Agent': 'HwAPI/0.0.1'}

 ### Important: This access Token will expire after 2 hours (or 1?), a new one has to be requested (permanent?) https://github.com/reddit-archive/reddit/wiki/OAuth2

In [9]:
res = requests.post('https://www.reddit.com/api/v1/access_token', 
                   auth=auth, data=user_data, headers=headers)

In [10]:
res.json()

{'access_token': '187569803486-n-lx_unVK9LbJC3K1WbU3wlD994vDQ',
 'token_type': 'bearer',
 'expires_in': 3600,
 'scope': '*'}

In [11]:
TOKEN = res.json()['access_token']

In [12]:
headers['Authorization'] = f'bearer {TOKEN}'

### Important: The API has a limit of requests per minute, monitor the usage. https://github.com/reddit-archive/reddit/wiki/API

In [13]:
res = requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

In [14]:
res.json()

{'is_employee': False,
 'seen_layout_switch': True,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': True,
 'pref_show_trending': True,
 'subreddit': {'default_set': True,
  'user_is_contributor': False,
  'banner_img': 'https://styles.redditmedia.com/t5_pra4k/styles/profileBanner_nxyt86yz98031.jpg?width=1280&amp;height=384&amp;crop=1280:384,smart&amp;s=ccbe7c186fd7e0e09acca5287daa3a4d8abd0d4f',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '',
  'user_is_muted': False,
  'display_name': 'u_Alepv',
  'header_img': None,
  'title': 'Alex',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://styles.redditmedia.com/t5_pra4k/styles/profileIcon_676bsq1r18031.jpeg?width=256&amp;height=256&amp;crop=256:256,smart&amp;s=43d1

In [15]:
sf_hot = requests.get('https://oauth.reddit.com/r/sanfrancisco/hot', headers=headers)

In [16]:
sf_hot.json()['data']

{'after': 't3_ow028i',
 'dist': 26,
 'modhash': None,
 'geo_filter': None,
 'children': [{'kind': 't3',
   'data': {'approved_at_utc': None,
    'subreddit': 'sanfrancisco',
    'selftext': 'Post about upcoming events, new things you’ve spotted around the city, or just little mundane sanfranciscoisms that strike your fancy. You can even do a little self-promotion here, if you abide by the rules in the sidebar. \n\n----\n\n* [Archive of previous daily discussions](https://www.reddit.com/r/sanfrancisco/search/?q=author%3Aautomoderator&amp;sort=new&amp;restrict_sr=on)\n* [Official San Francisco COVID-19 Data Tracker.](https://data.sfgov.org/stories/s/fjki-2fab) Complete with data &amp; easy to read charts &amp; graphs.\n* [Additional Covid info](https://covidactnow.org/us/ca/county/san_francisco_county?s=61890)',
    'author_fullname': 't2_6l4z3',
    'saved': False,
    'mod_reason_title': None,
    'gilded': 0,
    'clicked': False,
    'title': 'DAILY BULLSHIT — Sunday August 1, 2021',

In [17]:
import pandas as pd
from datetime import datetime
from time import sleep

In [18]:
data = pd.DataFrame()  # initialize dataframe
params = {'limit': 100}

In [19]:
# we use this function to convert responses to dataframes
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = df.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        }, ignore_index=True)

    return df

In [20]:
# loop through 10 times (returning 1K posts)
for i in range(3):
    # make request
    res = requests.get("https://oauth.reddit.com/r/sanfrancisco/new",
                       headers=headers,
                       params=params)

    # get dataframe from response
    new_df = df_from_response(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # create fullname
    fullname = row['kind'] + '_' + row['id']
    # add/update fullname in params
    params['after'] = fullname
    
    # append new_df to data
    data = data.append(new_df, ignore_index=True)
    
    sleep(1)

In [21]:
data

,created_utc,downs,id,kind,link_flair_css_class,score,selftext,subreddit,title,ups,upvote_ratio
0,2021-08-02T00:13:49Z,0.0,ow1lcj,t3,None,1.0,,sanfrancisco,S.F. temps didn't crack the 70s in July for fi...,1.0,1.00
1,2021-08-02T00:02:20Z,0.0,ow1dwo,t3,None,0.0,Stolen question from a different sub. But bas...,sanfrancisco,What are the most authentic non-‘Californian c...,0.0,0.25
2,2021-08-01T23:09:22Z,0.0,ow0fc7,t3,None,0.0,"Hey everyone, i have an opportunity to rent a ...",sanfrancisco,Living right next to Costco?,0.0,0.50
3,2021-08-01T22:49:58Z,0.0,ow028i,t3,pic,0.0,,sanfrancisco,PSA: Avoid Castro betw 19th and Elizabeth if y...,0.0,0.44
4,2021-08-01T22:28:47Z,0.0,ovzo73,t3,pic,3.0,,sanfrancisco,Got the growler filled yesterday for the first...,3.0,0.67
...,...,...,...,...,...,...,...,...,...,...,...
295,2021-07-24T01:23:49Z,0.0,oqe3go,t3,None,21.0,"It shows up on Google Maps, I checked it out i...",sanfrancisco,"Anybody have any information about the ""Giving...",21.0,0.87
296,2021-07-23T23:22:28Z,0.0,oqbukt,t3,None,15.0,Does anyone have any recommendations for a sho...,sanfrancisco,Recs for installing catalytic converter shield...,15.0,0.82
297,2021-07-23T22:27:06Z,0.0,oqargm,t3,None,23.0,,sanfrancisco,Eight-story condo overhaul of the Grubstake Di...,23.0,0.79
298,2021-07-23T22:26:22Z,0.0,oqaqwz,t3,None,10.0,Looking for submarine style bread for sandwich...,sanfrancisco,Best french roll bread to buy for sandwiches?,10.0,0.75


In [22]:
post_id = "ovq541"

In [23]:
res = requests.get(f"https://oauth.reddit.com/r/sanfrancisco/comments/{post_id}",
                   headers=headers,
                   params=params)

In [30]:
single_comment = res.json()[1]['data']['children'][0]
single_comment

{'kind': 't1',
 'data': {'total_awards_received': 0,
  'approved_at_utc': None,
  'author_is_blocked': False,
  'comment_type': None,
  'awarders': [],
  'mod_reason_by': None,
  'banned_by': None,
  'ups': 8,
  'author_flair_type': 'richtext',
  'removal_reason': None,
  'link_id': 't3_ovq541',
  'author_flair_template_id': None,
  'likes': None,
  'replies': {'kind': 'Listing',
   'data': {'after': None,
    'dist': None,
    'modhash': None,
    'geo_filter': '',
    'children': [{'kind': 't1',
      'data': {'total_awards_received': 0,
       'approved_at_utc': None,
       'author_is_blocked': False,
       'comment_type': None,
       'awarders': [],
       'mod_reason_by': None,
       'banned_by': None,
       'ups': 9,
       'author_flair_type': 'richtext',
       'removal_reason': None,
       'link_id': 't3_ovq541',
       'author_flair_template_id': 'ebc9c89a-3ac7-11e3-832e-12313d18f999',
       'likes': None,
       'replies': '',
       'author_fullname': 't2_g6wrh',
   

In [31]:
comment_keys = single_comment['data'].keys()
comment_keys

dict_keys(['total_awards_received', 'approved_at_utc', 'author_is_blocked', 'comment_type', 'awarders', 'mod_reason_by', 'banned_by', 'ups', 'author_flair_type', 'removal_reason', 'link_id', 'author_flair_template_id', 'likes', 'replies', 'author_fullname', 'saved', 'id', 'banned_at_utc', 'mod_reason_title', 'gilded', 'archived', 'collapsed_reason_code', 'no_follow', 'author', 'can_mod_post', 'send_replies', 'parent_id', 'score', 'approved_by', 'report_reasons', 'author_premium', 'all_awardings', 'subreddit_id', 'body', 'edited', 'user_reports', 'author_flair_css_class', 'downs', 'is_submitter', 'collapsed', 'author_flair_richtext', 'author_patreon_flair', 'body_html', 'gildings', 'collapsed_reason', 'associated_award', 'stickied', 'subreddit_type', 'can_gild', 'top_awarded_type', 'author_flair_text_color', 'score_hidden', 'permalink', 'num_reports', 'locked', 'name', 'created', 'subreddit', 'author_flair_text', 'treatment_tags', 'created_utc', 'subreddit_name_prefixed', 'controversial